In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather

In [2]:
import glob
import os

In [3]:
#path for 11 models
path = "/home/mlopez/EXEC/tmax_annual_85/"

In [4]:
files = list(glob.glob(os.path.join(path,'*.*')))

In [5]:
files

['/home/mlopez/EXEC/tmax_annual_85/BNU-ESM_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/INM-CM4_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/CMCC-CMS_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/IPSL-CM5A-LR_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/IPSL-CM5B-LR_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/NorESM1-M_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/CanESM2_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/GFDL-ESM2M_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/HadGEM2-CC_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/MPI-ESM-LR_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/ACCESS1-3_rcp85_tx_mean_annual.nc']

# Create DF of all models

In [6]:
dflist = []

In [7]:
for file in files:
    df = pcdf.load_as_df(file)
    dflist.append(df)

/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds['time'] = index.to_datetimeindex()
/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds['time'] = index.to_datetimeindex()
/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between

In [8]:
dflist

[                                    tx_mean
 lat       lon        time                  
 66.623306 -89.045212 1950-01-01  261.465546
                      1951-01-01  260.108063
                      1952-01-01  262.473969
                      1953-01-01  262.028931
                      1954-01-01  260.705750
 ...                                     ...
 40.041039 -74.045807 2096-01-01  298.027527
                      2097-01-01  296.727692
                      2098-01-01  296.084808
                      2099-01-01  296.549194
                      2100-01-01  296.858734
 
 [11507106 rows x 1 columns],                                     tx_mean
 lat       lon        time                  
 66.623306 -89.045212 1950-01-01  262.232147
                      1951-01-01  261.559753
                      1952-01-01  262.615112
                      1953-01-01  262.848358
                      1954-01-01  264.110535
 ...                                     ...
 40.041039 -74.045807 20

In [9]:
#Takes 3 min
%time df11 = pd.concat(dflist)

CPU times: user 4min 1s, sys: 12min 46s, total: 16min 47s
Wall time: 1min 33s


In [10]:
df11

tx_mean
lat       lon        time                  
66.623306 -89.045212 1950-01-01  261.465546
                     1951-01-01  260.108063
                     1952-01-01  262.473969
                     1953-01-01  262.028931
                     1954-01-01  260.705750
...                                     ...
40.041039 -74.045807 2096-01-01  295.437378
                     2097-01-01  294.912170
                     2098-01-01  296.226624
                     2099-01-01  295.801758
                     2100-01-01  296.634033

[126578166 rows x 1 columns]

In [11]:
dftgm = df11.reset_index()

In [12]:
dftgm

,lat,lon,time,tx_mean
0,66.623306,-89.045212,1950-01-01,261.465546
1,66.623306,-89.045212,1951-01-01,260.108063
2,66.623306,-89.045212,1952-01-01,262.473969
3,66.623306,-89.045212,1953-01-01,262.028931
4,66.623306,-89.045212,1954-01-01,260.705750
...,...,...,...,...
126578161,40.041039,-74.045807,2096-01-01,295.437378
126578162,40.041039,-74.045807,2097-01-01,294.912170
126578163,40.041039,-74.045807,2098-01-01,296.226624
126578164,40.041039,-74.045807,2099-01-01,295.801758


# Get the mean for 11 models

In [ ]:
#Takes 48s
%time dftgm2 = dftgm.groupby(["lat", "lon", "time"]).mean()

In [ ]:
dftgm2

# Transform temp to Celsius

In [ ]:
dfC = dftgm2.copy()
dfC["tx_mean"] = dfC["tx_mean"] -273.15
    

# Define 3 periods of time: 

In [ ]:
year_groups = {y:0 for y in range(1980,2011)}
year_groups.update({y:1 for y in range(2041,2071)})
year_groups.update({y:2 for y in range(2071,2101)})
#Get the mean of every period
dfC2 = dfC.reset_index()
dfp = dfC2.groupby([dfC2.time.dt.year.map(year_groups), "lat","lon"]).mean()

In [ ]:
dfp.reset_index()

# Historic Period

485,881/3 = 161,960. ---- instead 333,469 rows × 4 columns

In [ ]:
df_h = dfp.query("time==0.0")
df_h.reset_index()

In [ ]:
df2050 = dfp.query("time==1.0")
df2050.reset_index()

In [ ]:
df2080 = dfp.query("time==2.0")
df2080.reset_index()

## Import dataframe with polygons

In [ ]:
dfpolyshape = from_geofeather('Grid-TerritoiresGuides.feather')

In [ ]:
dfpolyshape

## Merge data with mask 

In [ ]:
dftg11TG = pd.merge(df_h, dfpolyshape, on=["lat","lon"])

In [ ]:
dftg11TG

In [ ]:
dftg11TG.TER_GUIDE.value_counts()

In [ ]:
TG4a = dftg11TG[dftg11TG.TER_GUIDE == "4a"]
TG4a

In [ ]:
TG4a.to_csv("/home/mlopez/EXEC/Tg_csv/TG4a11.csv")

# ---- Create GeoJson file ----

In [ ]:
geometry = dftg11TG["geometry"]

In [ ]:
crs = {'init': "epsg:4326"}
gdf = GeoDataFrame(dftg11TG, crs=crs, geometry=geometry)

In [ ]:
gdf.to_file("TG11tg.json", driver="GeoJSON")